In [51]:
import torch
import csv
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import os 
import sys
import numpy as np
from data_loader import DataOrganizer
from torchvision import transforms

In [52]:
model = torch.load("resnet50.pth", weights_only=False, map_location="cuda")
model.cuda()
thresholds = np.load("thresholds.npy")
thresholds

array([0.46292585, 0.44288577, 0.38877756, 0.52104208, 0.39078156,
       0.5       , 0.6252505 , 0.57915832, 0.70941884, 0.5       ,
       0.5       , 0.19238477, 0.5       , 0.53507014, 0.58116232,
       0.5       , 0.35871743, 0.51302605, 0.39278557, 0.37274549,
       0.5       , 0.47695391, 0.36472946, 0.27855711, 0.51703407,
       0.3507014 , 0.58917836, 0.01603206])

In [68]:
#means = [0.05273682624101639, 0.05463362857699394, 0.0804334506392479, 0.08266294747591019]
#sds = [0.11251822113990784, 0.15588141977787018, 0.14967259764671326,0.149698406457901] 

means = [0.04534007981419563, 0.040644969791173935, 0.05908305197954178, 0.05922728404402733]
sds = [0.10485349595546722, 0.13168220221996307, 0.12821660935878754,0.1246071457862854] 

transform = transforms.Compose([
    transforms.Normalize(mean=means, std=sds)
    ])

df = pd.read_csv("sample_submission.csv")
ids = df["Id"].values
IMG_PATH = "Data/test"
test_set = DataOrganizer(ids, IMG_PATH, transform=transform)
imgs = DataLoader(test_set, batch_size=8, shuffle=False, num_workers=4, pin_memory=True, persistent_workers=True)

In [69]:
# one time use to compute means and STDs
sum_ = torch.zeros(4, device="cuda")
sum_sq = torch.zeros(4, device="cuda")
pixels = 512*512*len(test_set)

for img in tqdm(imgs, desc="normalization stats"):
    img = img.cuda() 
    flat = img.view(8, 4, -1)
    sum_ += flat.sum(dim=(0, 2))        
    sum_sq += (flat ** 2).sum(dim=(0, 2))

mean = sum_ / pixels
std = ((sum_sq / pixels) - mean**2).sqrt()
mean = mean.cpu().numpy()
std = std.cpu().numpy()

for c in range(4):
    print(f"Channel: {c} | Mean: {mean[c]} | Std: {std[c]}")

normalization stats: 100%|█████████████████████████████████████████████████████████| 1463/1463 [02:00<00:00, 12.10it/s]

Channel: 0 | Mean: -3.7677997170249e-05 | Std: 0.9999059438705444
Channel: 1 | Mean: -4.3145966628799215e-05 | Std: 1.0000123977661133
Channel: 2 | Mean: 2.9918293876107782e-05 | Std: 1.0000098943710327
Channel: 3 | Mean: 3.043121614609845e-05 | Std: 0.9999994039535522


In [70]:
model.eval()
predictions = []
pb = tqdm(imgs, desc="Inference [Submission]")
with torch.no_grad():
    for img in pb:
        img = img.cuda(non_blocking=True)
        logits = model(img)
        probs  = torch.sigmoid(logits).cpu().numpy()
        predictions.append(probs)
predictions = np.vstack(predictions)

Inference [Submission]: 100%|██████████████████████████████████████████████████████| 1463/1463 [01:51<00:00, 13.18it/s]


In [71]:
res = [""]*predictions.shape[0]
predictions = (predictions > 0.5).astype(int)
for i in range(predictions.shape[0]):
    pred = predictions[i]
    x = ""
    for c in range(len(pred)):
        if pred[c] == 1:
            x+= str(c) + " "
    res[i] = x.strip() # formatting into strings for each 28-length class indicator array

In [72]:
submission_df = pd.DataFrame({"Id": ids, "Predicted": res})
submission_df.to_csv("submissions/submission2.csv", index=False)

In [67]:
df1 = pd.read_csv("sample_submission.csv")
df2 = pd.read_csv("submissions/submission1.csv")
df1["Id"].equals(df2["Id"])

True